In [25]:
import sys
import os
import ifcopenshell as _ifc

In [17]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

course_path = os.path.join(project_root, 'Course')
if course_path not in sys.path:
    sys.path.append(course_path)

from _general import mguu_cource_tools

In [18]:
example_file_path = os.path.join(project_root, 'DataExamples', 'Renga_House.ifc')
ifc_file = _ifc.open(example_file_path)

In [19]:
def read_object_properties(ifc_file, ifc_entity):
    out_props = dict()
    #IfcRelDefinesByProperties
    ifc_root_props = ifc_entity.IsDefinedBy
    for props_group in ifc_root_props:
        #print(props_group)
        props_definition = props_group.RelatingPropertyDefinition
        #print(props_definition)
        if props_definition.is_a("IfcPropertySet"):
            #print("IfcPropertySet")
            for props_definition_prop in props_definition.HasProperties:
                print(props_definition_prop)
                if props_definition_prop.is_a("IfcPropertySingleValue"):
                    out_props[props_definition_prop.Name] = props_definition_prop.NominalValue
                    #print(str(props_definition_prop.Name) + ' ' + str(props_definition_prop.NominalValue))
        elif props_definition.is_a("IfcElementQuantity"):
            #print("IfcElementQuantity")
            for one_quantity in props_definition.Quantities:
                #print(one_quantity)
                if one_quantity.is_a("IfcQuantityArea"):
                    out_props[one_quantity.Name] = one_quantity.AreaValue
                elif one_quantity.is_a("IfcQuantityCount"):
                    out_props[one_quantity.Name] = one_quantity.CountValue
                elif one_quantity.is_a("IfcQuantityLength"):
                    out_props[one_quantity.Name] = one_quantity.LengthValue
                elif one_quantity.is_a("IfcQuantityTime"):
                    out_props[one_quantity.Name] = one_quantity.TimeValue
                elif one_quantity.is_a("IfcQuantityVolume"):
                    out_props[one_quantity.Name] = one_quantity.VolumeValue
                elif one_quantity.is_a("IfcQuantityWeight"):
                    out_props[one_quantity.Name] = one_quantity.WeightValue
    return out_props

ifc_railing_id = '3ZZpg7C_CNTUvpxzN75M8Y'
ifc_railing = ifc_file.by_guid(ifc_railing_id)
object_props = read_object_properties(ifc_file=ifc_file, ifc_entity=ifc_railing)

object_props

#23280=IfcPropertySingleValue('FireRating',$,$,$)
#23281=IfcPropertySingleValue('Reference',$,IfcLabel('Колонна 200x200 мм, L=2900'),$)
#23282=IfcPropertySingleValue('Roll',$,IfcPlaneAngleMeasure(0.),#16)
#23285=IfcPropertySingleValue('ReinforcementBarType',$,IfcLabel('(нет)'),$)


{'FireRating': None,
 'Reference': IfcLabel('Колонна 200x200 мм, L=2900'),
 'Roll': IfcPlaneAngleMeasure(0.),
 'ReinforcementBarType': IfcLabel('(нет)'),
 'CrossSectionArea': 0.04,
 'Length': 3000.0,
 'NetVolume': 0.112664442783435,
 'NetWeight': 77.73846552057,
 'OuterSurfaceArea': 2.253288855668695}

In [20]:
def get_object_geometry(ifc_entity):
    object_Representation = ifc_entity.Representation
    print(object_Representation)
    if object_Representation.is_a("IfcProductDefinitionShape"):
        object_Representations = object_Representation.Representations
        for object_Representation_one in object_Representations:
            print(object_Representation_one)
            if object_Representation_one.is_a("IfcShapeRepresentation"):
                items = object_Representation_one.Items
                for item in items:
                    print(item)
                    if item.is_a("IfcMappedItem"):
                        shape = item.MappingSource.MappedRepresentation
                        print(shape)
                        for sub_item in shape.Items:
                            print(sub_item)
                            if sub_item.is_a("IfcFacetedBrep"):
                                for one_face in sub_item.Outer.CfsFaces:
                                    print(one_face.Bounds[0].Bound)
                                    for one_plg in one_face.Bounds[0].Bound.Polygon:
                                        print(one_plg)


ifc_column_id = '3ZZpg7C_CNTUvpxzN75M8Y'
ifc_column = ifc_file.by_guid(ifc_column_id)

geometry_props = get_object_geometry(ifc_column)

geometry_props

#12465=IfcProductDefinitionShape($,$,(#12464))
#12464=IfcShapeRepresentation(#41,'Body','MappedRepresentation',(#12463))
#12463=IfcMappedItem(#12457,#12462)
#12452=IfcShapeRepresentation(#41,'Body','Brep',(#12451))
#12451=IfcFacetedBrep(#12450)
#12432=IfcPolyLoop((#12382,#12383,#12384,#12385))
#12382=IfcCartesianPoint((-100.,-100.,2900.521032703575202))
#12383=IfcCartesianPoint((-100.,100.,2732.701106468118724))
#12384=IfcCartesianPoint((-100.,100.,0.))
#12385=IfcCartesianPoint((-100.,-100.,0.))
#12435=IfcPolyLoop((#12383,#12389,#12390,#12384))
#12383=IfcCartesianPoint((-100.,100.,2732.701106468118724))
#12389=IfcCartesianPoint((100.,100.,2732.701106468120543))
#12390=IfcCartesianPoint((100.,100.,0.))
#12384=IfcCartesianPoint((-100.,100.,0.))
#12438=IfcPolyLoop((#12389,#12394,#12395,#12390))
#12389=IfcCartesianPoint((100.,100.,2732.701106468120543))
#12394=IfcCartesianPoint((100.,-100.,2900.521032703576566))
#12395=IfcCartesianPoint((100.,-100.,0.))
#12390=IfcCartesianPoint((100.,100.,

In [21]:
def get_info_by_storeys_of_class(ifcfile, ifc_class_name, property_name):
    out_info = dict()
    ifc_storeys = ifcfile.by_type("IfcBuildingStorey")
    ifc_storeys.sort(key= lambda a:a.Name)
    for one_storey in ifc_storeys:
        temp_prop_value = 0.0
        storey_objects = one_storey.ContainsElements[0].RelatedElements
        for storey_object in storey_objects:
            if storey_object.is_a(ifc_class_name):
                dict_props = mguu_cource_tools.get_object_properties(storey_object)
                prop_value = dict_props[property_name]
                temp_prop_value += prop_value
        out_info[one_storey.Name] = temp_prop_value
    return out_info

dict_storeys_volume = get_info_by_storeys_of_class(ifc_file, "IfcSlab", "NetVolume")
print(str(dict_storeys_volume))

{'1-й этаж': 71.05273929135467, '2-й этаж': 37.959999999999866, 'Кровля': 0.0}


In [24]:
#Перебор файлов в папке
root_path_os= os.path.join(project_root, 'DataExamples', 'ProjectEvolution')
files_ifc_list_temp = list()

all_files_temp = os.listdir(root_path_os)
for one_file in all_files_temp:
    #print(one_file)
    if one_file.endswith(".ifc"):
        #print(one_file)
        files_ifc_list_temp.append(os.path.join(root_path_os, one_file))

#Работаем с файлами
out_dict = dict()
for one_path in files_ifc_list_temp:
    #print(one_path)
    ifc_file = _ifc.open(one_path)
    file_name = os.path.basename(one_path)
    props_info = get_info_by_storeys_of_class(ifc_file, "IfcSlab", "NetVolume")
    temp_summa_value = 0.0
    for k,v in props_info.items():
        temp_summa_value += v
    out_dict[file_name] = temp_summa_value

out_dict

{'Renga_Structure-ver_1.ifc': 145.7650193359838,
 'Renga_Structure-ver_2.ifc': 222.17301933598378,
 'Renga_Structure-ver_3.ifc': 298.5810193359838,
 'Renga_Structure-ver_4.ifc': 374.9890193359838,
 'Renga_Structure-ver_5.ifc': 451.3970193359838,
 'Renga_Structure-ver_6.ifc': 527.8050193359838,
 'Renga_Structure-ver_7.ifc': 604.2130193359839,
 'Renga_Structure-ver_8.ifc': 680.6210193359839}